In [1]:
import sys; sys.path.append('..')
from os import path
from random import choice, shuffle

from graph import Graph, ReaderORLibrary
from graph.steiner import (prunning_mst, shortest_path,
                            shortest_path_origin_prim,
                            prunning_kruskal_mst,
                            shortest_path_with_origin)

from graph.util import is_steiner_tree, has_cycle
from graph.algorithms import kruskal, prim

In [2]:
from graph.util import gg_total_weight

In [3]:
# input
dataset_file = 'steinc5.txt'

# output
csv_output = 'resultado.csv'
graphs_output = 'grafos.pickle'

In [4]:
file = path.join('..', 'datasets','ORLibrary', dataset_file)

assert path.exists(file)

reader = ReaderORLibrary()

stpg = reader.parser(file)

In [5]:
from prunningcrossover import PrunningCrossover

In [6]:
crossover = PrunningCrossover(stpg)

In [7]:
vertices = list(stpg.graph.vertices)

shuffle(vertices)

print(vertices[:10],' ...')

[62, 433, 393, 299, 179, 211, 120, 355, 455, 291]  ...


In [8]:
v = vertices.pop()

aa, aa_cost = shortest_path_with_origin(stpg.graph, v, stpg.terminals)

r, test = is_steiner_tree(aa, stpg)

r

False

In [9]:
test

{'has_cycle': False,
 'all_terminals_in': True,
 'all_leaves_are_terminals': False,
 'all_edges_are_reliable': True,
 'graph_is_connected': True}

In [11]:
%%time

data = list()

for _ in range(200):
    v = vertices.pop()
    aa, aa_cost = prunning_mst(stpg.graph, v, stpg.terminals)
    is_aa_st, _ = is_steiner_tree(aa, stpg)
    
    u = vertices.pop()
    bb, bb_cost = prunning_mst(stpg.graph, u, stpg.terminals)
    is_bb_st, _ = is_steiner_tree(bb, stpg)
    
    child = crossover(aa, bb)
    cc_cost = gg_total_weight(child)
    is_cc_st, _ = is_steiner_tree(child, stpg)
    
    data.append([v, u, is_aa_st, is_bb_st, is_cc_st, aa_cost, bb_cost, cc_cost])

Wall time: 14.3 s


In [57]:
# child.edges

In [58]:
import pandas as pd

In [59]:
frame = pd.DataFrame(data, columns=['start_a', 'start_b', 'is_aa_st', 'is_bb_st', 'is_cc_st' ,'a_cost', 'b_cost', 'c_cost'])

In [60]:
frame['less_both'] = (frame['c_cost'] < frame['a_cost']) & (frame['c_cost'] < frame['b_cost'])

In [61]:
frame['less_equal_both'] = (frame['c_cost'] <= frame['a_cost']) & (frame['c_cost'] <= frame['b_cost'])

In [62]:
frame['less_a'] = (frame['c_cost'] <= frame['a_cost'])

In [63]:
frame['less_b'] = (frame['c_cost'] <= frame['b_cost'])

In [64]:
frame.head()

,start_a,start_b,is_aa_st,is_bb_st,is_cc_st,a_cost,b_cost,c_cost,less_both,less_equal_both,less_a,less_b
0,284,147,True,True,True,1638,1640,1638,False,True,True,True
1,455,492,True,True,True,1640,1640,1640,False,True,True,True
2,308,287,True,True,True,1640,1634,1640,False,False,True,False
3,34,93,True,True,True,1637,1634,1634,False,True,True,True
4,361,302,True,True,True,1637,1636,1634,True,True,True,True


In [65]:
frame['is_aa_st'].unique()

array([ True])

In [66]:
frame['is_bb_st'].unique()

array([ True])

In [67]:
frame['is_cc_st'].unique()

array([ True])

In [68]:
frame[['a_cost', 'b_cost','c_cost']].min()

a_cost    1630
b_cost    1630
c_cost    1630
dtype: int64

In [69]:
frame[['a_cost', 'b_cost','c_cost']].max()

a_cost    1652
b_cost    1648
c_cost    1648
dtype: int64

In [70]:
frame[['less_both', 'less_a', 'less_b', 'less_equal_both']].describe()

,less_both,less_a,less_b,less_equal_both
count,200,200,200,200
unique,2,2,2,2
top,False,True,True,True
freq,182,150,149,108
